### Imports

In [1]:
import os
from glob import glob
from PIL import Image
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

### Evaluate Original Images

In [2]:
image_path = '../../Data.nosync/Zalando_Germany_Dataset/dresses/images/raw_images'
all_images = glob(image_path + '/*')
print(f"Found {len(all_images)} images")

Found 14060 images


### Check Image Sizes:

In [3]:
sizes = {}
for image in tqdm(all_images):
    img = Image.open(image)
    sizes[image.split('/')[-1]] = img.size
sizes = pd.Series(sizes)
sizes.value_counts()

  0%|          | 0/14060 [00:00<?, ?it/s]

(1024, 1478)    5885
(1024, 1475)    4348
(1024, 1474)    1548
(1024, 1479)    1169
(900, 1296)      275
                ... 
(663, 955)         1
(808, 1164)        1
(795, 1144)        1
(831, 1196)        1
(973, 1401)        1
Length: 139, dtype: int64

### Making Images Square
- Since dresses have an aspect ratio which is longer than wide, most images are not square but rectangular. 
- In the following, I will make them square by adding white padding to the shorter side of the image.
- This is done symmetrically such that the item is still centered in the image.


In [4]:
def make_square(im, resize_size = None):
    max_size = max(im.size)
    # Infer bckground color from top left pixel
    background_color = im.getpixel((0,0))
    # Create new image with correct background
    new_im = Image.new("RGB", (max_size, max_size), color=background_color)
    # Paste image in center
    new_im.paste(im, box = ((max_size - im.size[0])//2,0))

    # If Resizing so specific size is needed:
    if resize_size:
        new_im = new_im.resize((resize_size, resize_size))

    return new_im

### Saving Square Images
- The square images are saved to a new directory
- For now, the images will be resized to 512x512 pixels. Depending on the model that is trained, this can be adapted


In [5]:
save_dir = '../../Data.nosync/Zalando_Germany_Dataset/dresses/images/square_images'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Flag whether to reproduce the square images. If False, the code will skip images that already exist. Needs to be set to True if you want to create sqaures of another size
reproduce = False

for image in tqdm(all_images):
    # Skip if already exists
    if os.path.exists(save_dir + '/' + image.split('/')[-1]) and not reproduce:
        continue
    else:
        img = Image.open(image)
        img = make_square(img, resize_size = 1024)
        img.save(save_dir + '/' + image.split('/')[-1])

  0%|          | 0/14060 [00:00<?, ?it/s]

### Run StyleGAN2-Ada Image Preprocessing

In [6]:
CMD = "python ../stylegan2-ada-pytorch/dataset_tool.py "\
  "--source ../../Data.nosync/Zalando_Germany_Dataset/dresses/images/square_images "\
  "--dest ../../Data.nosync/Zalando_Germany_Dataset/dresses/images/stylegan2_ada_images" \
  " --width=512 --height=512"

!{CMD}

../stylegan2-ada-pytorch/dataset_tool.py:205: DeprecationWarning: BOX is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BOX instead.
  resample = { 'box': PIL.Image.BOX, 'lanczos': PIL.Image.LANCZOS }[resize_filter]
../stylegan2-ada-pytorch/dataset_tool.py:205: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  resample = { 'box': PIL.Image.BOX, 'lanczos': PIL.Image.LANCZOS }[resize_filter]
100%|█████████████████████████████████████| 14060/14060 [13:34<00:00, 17.27it/s]
